In [41]:
class FixedPoint:

    def check_value_range(self,value):
        if value > self.max_value or value < self.min_value:
            raise ValueError(f"Value {value} cannot be represented with precision {self.precision} max_val:{self.max_value} min_val:{self.min_value}")

    def recompute_value_according_to_precision(self,value):
        self.check_value_range(value)
        int_bits, frac_bits = self.precision
        scale = 2 ** frac_bits
        scaled_value = int(value * scale)
        rounded_value = round(scaled_value)
        return rounded_value / scale

    def __init__(self, value, precision,signed=True):
        sub1 = 1 if signed else 0
        self.max_value = (2**(precision[0]+precision[1] - sub1)-1) / 2**(precision[1])
        self.min_value = 0 if not signed else -self.max_value
        self.signed = signed
        self.precision = precision
        self.value = self.recompute_value_according_to_precision(value)
    
    def __str__(self):
        sign = "-" if self.value < 0 else ""
        signed = "SIGNED" if self.signed else "UNSIGNED"
        int_part = abs(int(self.value))
        frac_part = abs(self.value) - int_part
        int_str = bin(int_part)[2:].zfill(self.precision[0])
        frac_str = ""
        for _ in range(self.precision[1]):
            frac_part *= 2
            bit = int(frac_part)
            frac_str += str(bit)
            frac_part -= bit
        return f"signed {sign}{int_str}.{frac_str} value:{self.value} max_val:{self.max_value} min_val:{self.min_value}"

    def __repr__(self):
        return f'FixedPoint({self.value}, {self.precision}) {self.__str__()}'

    
    def __add__(self, other):
        if not isinstance(other, FixedPoint):
            raise TypeError('unsupported operand type(s) for +: \'FixedPoint\' and \'' + type(other).__name__ + '\'')
        if self.precision != other.precision:
            raise ValueError('operands have different precision')
        return FixedPoint(self.value + other.value, self.precision)
    
    def __sub__(self, other):
        if not isinstance(other, FixedPoint):
            raise TypeError('unsupported operand type(s) for -: \'FixedPoint\' and \'' + type(other).__name__ + '\'')
        if self.precision != other.precision:
            raise ValueError('operands have different precision')
        return FixedPoint(self.value - other.value, self.precision)
    
    def __mul__(self, other):
        if isinstance(other, (int, float)):
            return FixedPoint(self.value * other, self.precision)
        elif isinstance(other, FixedPoint):
            return FixedPoint((self.value*other.value), (self.precision[0] + other.precision[0],self.precision[1] + other.precision[1]))
        else:
            raise TypeError('unsupported operand type(s) for *: \'FixedPoint\' and \'' + type(other).__name__ + '\'')
    
    def __truediv__(self, other):
        if isinstance(other, (int, float)):
            return FixedPoint(self.value / other, self.precision)
        if isinstance(other, FixedPoint):
            return FixedPoint((self.value/other.value), (other.precision[0],self.precision[1] - other.precision[1]))
        else: 
            raise TypeError('unsupported operand type(s) for /: \'FixedPoint\' and \'' + type(other).__name__ + '\'')
    def __rmul__(self, other):
        if not isinstance(other, (int, float)):
            raise TypeError('unsupported operand type(s) for *: \'' + type(other).__name__ + '\' and \'FixedPoint\'')
        return FixedPoint(self.value * other, self.precision)
    
    def __eq__(self, other):
        if not isinstance(other, FixedPoint):
            return False
        return self.value == other.value and self.precision == other.precision
    
    def __ne__(self, other):
        return not self.__eq__(other)


In [51]:
a = FixedPoint(0.8, (8, 8))
b = FixedPoint(0.08, (8, 8))
print(a,b)
c = a / b
print(a,b,c)
#print (a/2)

signed 00000000.11001100 value:0.796875 max_val:127.99609375 min_val:-127.99609375 signed 00000000.00010100 value:0.078125 max_val:127.99609375 min_val:-127.99609375
signed 00000000.11001100 value:0.796875 max_val:127.99609375 min_val:-127.99609375 signed 00000000.00010100 value:0.078125 max_val:127.99609375 min_val:-127.99609375 signed 00001010. value:10.0 max_val:127.0 min_val:-127.0


In [48]:
bin(int(0.05*(2**4)))


'0b0'

In [33]:
q=(1,2)
d=(3,4)
print(q+d)

(1, 2, 3, 4)


In [37]:
import unittest

class TestFixedPoint(unittest.TestCase):
    
    def test_subtraction(self):
        a = FixedPoint(0.5, (3, 2))
        b = FixedPoint(-0.75, (3, 2))
        c = a - b
        self.assertEqual(c.value, 1.25)
        self.assertEqual(c.precision, (3, 2))
        
    def test_multiplication(self):
        a = FixedPoint(0.5, (3, 2))
        b = FixedPoint(-0.75, (3, 2))
        c = a * b
        self.assertEqual(c.value, -0.375)
        self.assertEqual(c.precision, (6, 4))
        
    def test_division(self):
        a = FixedPoint(0.5, (3, 2))
        b = FixedPoint(-0.75, (3, 2))
        c = a / b
        self.assertEqual(c.value, -0.6666666666666666)
        self.assertEqual(c.precision, (3, 0))
        
    def test_equal(self):
        a = FixedPoint(0.5, (3, 3))
        b = FixedPoint(0.5, (3, 3))
        c = FixedPoint(0.6, (3, 3))
        self.assertEqual(a, b)
        self.assertNotEqual(a, c)
        
    def test_str(self):
        a = FixedPoint(0.5, (3, 2))
        self.assertEqual(str(a), "000.1")

runner = unittest.TextTestRunner()
result = runner.run(unittest.makeSuite(TestFixedPoint))
print(result)


FF.F.

<unittest.runner.TextTestResult run=5 errors=0 failures=3>



FAIL: test_division (__main__.TestFixedPoint)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-37-548b8e48a80c>", line 23, in test_division
    self.assertEqual(c.value, -0.6666666666666666)
AssertionError: 0.0 != -0.6666666666666666

FAIL: test_equal (__main__.TestFixedPoint)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-37-548b8e48a80c>", line 31, in test_equal
    self.assertNotEqual(a, c)
AssertionError: FixedPoint(0.5, (3, 3)) signed 000.100 max_val:3.875 min_val:-3.875 == FixedPoint(0.5, (3, 3)) signed 000.100 max_val:3.875 min_val:-3.875

FAIL: test_str (__main__.TestFixedPoint)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-37-548b8e48a80c>", line 35, in test_str
    self.assertEqual(str(a), "000.1")
AssertionError: 'signed 000.10 max_v